## Tuning Hyperparameter for knn ContentBasedFiltering

Run: 

<i> '$' pipenv shell </i>

<i> '$' pipenv install ipynb </i>

to be able to import functions from other ipynb files


In [1]:
pip install ipynb


You should consider upgrading via the '/home/fx/.virtualenvs/bda-analytics-challenge-ss2020-UmK7jAse/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import os
import sys
from knn_preprocessing import knn_preprocessing
#Import will run notebook and perform PCA
from ipynb.fs.full.ContentBasedFiltering import predict_movie_rating, test_predict_mr, get_data

PCA:
Main Variance impacting factors:
[0.1352569  0.09503712 0.07998231 0.06074159 0.04796725 0.04597668
 0.0430597  0.04134133 0.03750373 0.03703565 0.03502601 0.0336219
 0.03295991 0.03155028 0.0294954  0.02847374 0.02764069 0.02351684
 0.02326137 0.01913027 0.01828914 0.01731806 0.0156969  0.01428033
 0.01294528 0.00832974 0.0045619 ]
     weights        features  abs_weights
2   0.460198      imdbRating     0.460198
5   0.428491      Metacritic     0.428491
4   0.422208  RottenTomatoes     0.422208
6   0.352119          Awards     0.352119
3   0.305617       imdbVotes     0.305617
1   0.277951         Runtime     0.277951
14  0.209256               7     0.209256
0  -0.148325            Year     0.148325
11 -0.143027               4     0.143027
25  0.120398              18     0.120398
17 -0.086768              10     0.086768
12  0.083021               5     0.083021
16  0.067442               9     0.067442
18  0.049997              11     0.049997
22 -0.047021              15  

In [3]:
def kreuzvalidiere(n, folds, mean = False, knn_metric = 'cosine', set_k=False, k_neighbors=15, data=get_data()):
    upper = 0
    lower = 10
    
    doku = pd.DataFrame(columns={'RMSE','Testsize','Fold','mean', 'knn_metric','set_k','k_neighbors','data'})
    
    
    for i in range(0,folds):
        rmse = test_predict_mr(n, mean, knn_metric, set_k, k_neighbors)
        doku.loc[i,'RMSE'] = rmse
        doku.loc[i,'Testsize'] = n
        doku.loc[i,'Fold'] = str(i)
        doku.loc[i,'mean'] = mean
        doku.loc[i,'knn_metric'] = knn_metric
        doku.loc[i,'set_k'] = set_k
        doku.loc[i,'data'] = str(data.columns)
        if set_k:
            doku['k_neighbors'] = k_neighbors
        
        
        if (float(rmse) < float(lower)):
            lower = rmse

        if (float(rmse) > float(upper)):
            upper = rmse

    print('RMSE upper Bound: '+str(upper))
    print('RMSE lower Bound: '+str(lower))
    
    #build output csv
    doku = doku.sort_values(by=['RMSE'], ascending=True)
    doku = doku.dropna(how='all', axis=1)
    doku.to_csv('../../data/tuning/ContentbasedTuning_'+str(round(lower,4))+'.csv') 

## Tuning for Mean() vs. distance-weighted rating

In [32]:
n = 100
folds = 4
print("Testing for n= "+str(n)+", Mean: False"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, mean=False)

Testing for n= 100, Mean: False, Folds: 4
RMSE: 0.804577548248386
RMSE: 0.8327401654270914
RMSE: 0.7110886702295112
RMSE: 0.8678981722445336
RMSE upper Bound: 0.8678981722445336
RMSE lower Bound: 0.7110886702295112


In [10]:
n = 25000
folds = 10
print("Testing for n= "+str(n)+", Mean: False"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, mean=False)

Testing for n= 25000, Mean: False, Folds: 10
RMSE: 0.8093717866581247
RMSE: 0.8109298741225405
RMSE: 0.8122008042821607
RMSE: 0.804284010080862
RMSE: 0.8091955357611241
RMSE: 0.8068427998071319
RMSE: 0.8090052843861771
RMSE: 0.8122322697998022
RMSE: 0.8059915436817127
RMSE: 0.8087654925187531
RMSE upper Bound: 0.8122322697998022
RMSE lower Bound: 0.804284010080862


In [11]:
n = 25000
folds = 10
print("Testing for n= "+str(n)+", Mean: True"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, mean=True)

Testing for n= 25000, Mean: True, Folds: 10
RMSE: 0.8104320862629049
RMSE: 0.812571557942916
RMSE: 0.8105755766394918
RMSE: 0.8158718554745932
RMSE: 0.8194891221568065
RMSE: 0.8190834153694807
RMSE: 0.8160188609612186
RMSE: 0.8127711506527119
RMSE: 0.8084195815842717
RMSE: 0.8103183042621105
RMSE upper Bound: 0.8194891221568065
RMSE lower Bound: 0.8084195815842717


## Tuning for knn_metric

In [4]:
n = 100
folds = 4
print("Testing for n= "+str(n)+", Metric: 'cosine'"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, knn_metric = 'cosine')

Testing for n= 100, Metric: 'cosine', Folds: 4
RMSE: 0.9280154286780048
RMSE: 0.7608646440927771
RMSE: 0.8010978236619253
RMSE: 0.7112706775734763
RMSE upper Bound: 0.9280154286780048
RMSE lower Bound: 0.7112706775734763


In [5]:
n = 25000
folds = 10
print("Testing for n= "+str(n)+", Metric: 'cosine'"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, knn_metric = 'cosine')

Testing for n= 25000, Metric: 'cosine', Folds: 10
RMSE: 0.8180188820765721
RMSE: 0.8042295061506927
RMSE: 0.8122801216107732
RMSE: 0.8098929481095994
RMSE: 0.8139925754947308
RMSE: 0.8022085441161241
RMSE: 0.8110411796668742
RMSE: 0.8065866699637653
RMSE: 0.8079964316907624
RMSE: 0.8067369339424777
RMSE upper Bound: 0.8180188820765721
RMSE lower Bound: 0.8022085441161241


In [6]:
n = 25000
folds = 10
print("Testing for n= "+str(n)+", Metric: 'minkowski'"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, knn_metric = 'minkowski')

Testing for n= 25000, Metric: 'minkowski', Folds: 10
RMSE: 0.8165255607397872
RMSE: 0.8080976788471829
RMSE: 0.8114821154932244
RMSE: 0.8152800252936769
RMSE: 0.8129823727637878
RMSE: 0.810843060323114
RMSE: 0.8090826533626234
RMSE: 0.8176513974631802
RMSE: 0.8136693410726308
RMSE: 0.8211539453639878
RMSE upper Bound: 0.8211539453639878
RMSE lower Bound: 0.8080976788471829


## Tuning for k_neighbors

In [7]:
n = 100
folds = 4
print("Testing for n= "+str(n)+", set_k: False"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, set_k=False)

Testing for n= 100, set_k: False, Folds: 4
RMSE: 0.8667067213604475
RMSE: 0.8363389438753414
RMSE: 0.8891811447302662
RMSE: 0.9472135760668952
RMSE upper Bound: 0.9472135760668952
RMSE lower Bound: 0.8363389438753414


In [8]:
n = 25000
folds = 10
print("Testing for n= "+str(n)+", set_k: False"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, set_k=False)

Testing for n= 25000, set_k: False, Folds: 10
RMSE: 0.8082220020173763
RMSE: 0.8103171667364057
RMSE: 0.8152772859897605
RMSE: 0.8012471381544237
RMSE: 0.8174343129763451
RMSE: 0.8031949602814995
RMSE: 0.8089629761973997
RMSE: 0.8054906122933797
RMSE: 0.8145465406878625
RMSE: 0.8142678494344093
RMSE upper Bound: 0.8174343129763451
RMSE lower Bound: 0.8012471381544237


In [12]:
n = 25000
folds = 10
for i in range(4,10):
    print("Testing for n= "+str(n)+", set_k: True,"+"k= "+str(i)+ "Folds: "+str(folds))
    kreuzvalidiere(n, folds, set_k=True, k_neighbors=i)

Testing for n= 25000, set_k: True,k= 4Folds: 10
RMSE: 0.8079389664944242
RMSE: 0.8115115045295346
RMSE: 0.8080101908819916
RMSE: 0.8127231024410206
RMSE: 0.8117074853671281
RMSE: 0.8074752738658302
RMSE: 0.8174509720347846
RMSE: 0.8146010913328614
RMSE: 0.8065443689966026
RMSE: 0.8172577543320944
RMSE upper Bound: 0.8174509720347846
RMSE lower Bound: 0.8065443689966026
Testing for n= 25000, set_k: True,k= 5Folds: 10
RMSE: 0.8062363195831578
RMSE: 0.806536109705288
RMSE: 0.8137355707213381
RMSE: 0.8089176442774695
RMSE: 0.815513378259798
RMSE: 0.8048125321726615
RMSE: 0.8085902478740857
RMSE: 0.8127757835161609
RMSE: 0.8076366701210245
RMSE: 0.8108814619888898
RMSE upper Bound: 0.815513378259798
RMSE lower Bound: 0.8048125321726615
Testing for n= 25000, set_k: True,k= 6Folds: 10
RMSE: 0.8132573293637523
RMSE: 0.8094053095701897
RMSE: 0.8180984463335088
RMSE: 0.8147020409645342
RMSE: 0.8083418924377065
RMSE: 0.8042591459822069
RMSE: 0.7986141815660582
RMSE: 0.8083104847373419
RMSE: 0.816

In [13]:
n = 25000
folds = 10
for i in range(10,20):
    print("Testing for n= "+str(n)+", set_k: True,"+"k= "+str(i)+ "Folds: "+str(folds))
    kreuzvalidiere(n, folds, set_k=True, k_neighbors=i)

Testing for n= 25000, set_k: True,k= 10Folds: 10
RMSE: 0.8106622351248536
RMSE: 0.8088263802145101
RMSE: 0.81423633718393
RMSE: 0.8007395611400208
RMSE: 0.810220133131492
RMSE: 0.8009805468464142
RMSE: 0.8061972898129933
RMSE: 0.8035640854092879
RMSE: 0.8041458166678931
RMSE: 0.808795687562556
RMSE upper Bound: 0.81423633718393
RMSE lower Bound: 0.8007395611400208
Testing for n= 25000, set_k: True,k= 11Folds: 10
RMSE: 0.809646992549816
RMSE: 0.8117489254688095
RMSE: 0.8070131032906482
RMSE: 0.8108917317065666
RMSE: 0.8092727220721694
RMSE: 0.8101591240369034
RMSE: 0.8137420506847454
RMSE: 0.8136439808418129
RMSE: 0.8049082261292966
RMSE: 0.8045675529820194
RMSE upper Bound: 0.8137420506847454
RMSE lower Bound: 0.8045675529820194
Testing for n= 25000, set_k: True,k= 12Folds: 10
RMSE: 0.8121929230452263
RMSE: 0.8136741095204966
RMSE: 0.8125351546169084
RMSE: 0.8052722412753475
RMSE: 0.8070891538990275
RMSE: 0.8092939633823466
RMSE: 0.8145341190971557
RMSE: 0.8079342892208258
RMSE: 0.8132

In [14]:
n = 25000
folds = 10
for i in range(20,30)
    print("Testing for n= "+str(n)+", set_k: True,"+"k= "+str(i)+ "Folds: "+str(folds))
    kreuzvalidiere(n, folds, set_k=True, k_neighbors=i)

SyntaxError: invalid syntax (<ipython-input-14-04b734621bdf>, line 3)

## Tuning for input features

In [ ]:
data = get_data()

In [ ]:
data

In [ ]:
#TODO Functionen umschreiben so, dass man auch auf feature Hyperparameter tunen kann 
#->kreuzvalidiere und test_predict_mr müssen 'data' parameter bekommen

In [ ]:
n = 25000
folds = 10
data = data['Zu testende Feature Spalten eintragen']
print("Testing for n= "+str(n)+", Features: Tbd,"+ "Folds: "+str(folds))
kreuzvalidiere(n, folds, set_k=True, k_neighbors=i, data)